In [1]:
#RNN

In [5]:
import d2lzh as d2l
import math 
from mxnet import autograd,nd
from mxnet.gluon import loss as gloss
import time
import zipfile

#(corpus_indices,char_to_idx,idx_to_char,vocab_size) = d2l.load_data_jay_lyrics()
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices,char_to_idx,idx_to_char,vocab_size) = load_data_jay_lyrics()

In [6]:
nd.one_hot(nd.array([0,2]),vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [7]:
#将训练集进行onehot编码
def to_onehot(X,size):
    return [nd.one_hot(x,size) for x in X.T]

X = nd.arange(10).reshape((2,5))
inputs = to_onehot(X,vocab_size)
len(inputs),inputs[0].shape

(5, (2, 1027))

In [9]:
#初始化模型参数
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
ctx = d2l.try_gpu()
print('will use',ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01,shape=shape,ctx=ctx)
    
    #hidden layer
    W_xh = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    b_h = nd.zeros(num_hiddens,ctx=ctx)
    
    #output layer
    W_hq = _one((num_hiddens,num_outputs))
    b_q = nd.zeros(num_outputs,ctx=ctx)
    
    #grad
    params = [W_xh,W_hh,b_h,W_hq,b_q]
    for param in params:
        param.attach_grad()
    return params

will use gpu(0)


In [10]:
#model
def init_rnn_state(batch_size,num_hiddens,ctx):
    return (nd.zeros(shape=(batch_size,num_hiddens),ctx=ctx),)

In [11]:
def rnn(inputs,state,params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X,W_xh)+nd.dot(H,W_hh)+b_h)  #使用tanh作为激活函数
        Y = nd.dot(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [12]:
state = init_rnn_state(X.shape[0],num_hiddens,ctx)
inputs = to_onehot(X.as_in_context(ctx),vocab_size)
params = get_params()
outputs,state_new  = rnn(inputs,state,params)
len(outputs),outputs[0].shape,state_new[0].shape

(5, (2, 1027), (2, 256))

In [13]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,
                num_hiddens,vocab_size,ctx,idx_to_char,char_to_idx):
    state = init_rnn_state(1,num_hiddens,ctx)
    #基于前缀prefix（含有数个字符的字符串）预测
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        # 将上⼀时间步的输出作为当前时间步的输⼊
        X = to_onehot(nd.array([output[-1]],ctx=ctx),vocab_size)
        # 计算输出和更新隐藏状态
        (Y,state) = rnn(X,state,params)
        # 下⼀个时间步的输⼊是prefix⾥的字符或者当前的最佳预测字符
        if t <len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [14]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            ctx, idx_to_char, char_to_idx)

'分开瑰恩b痛银望杨搞靠星'

In [15]:
#剪裁梯度 防止梯度爆炸
def grad_clipping(params,theta,ctx):
    norm = nd.array([0],ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:   #梯度的L2范数不超过theta
        for param in params:
            param.grad[:] *= theta / norm

In [19]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,
                        ctx,corpus_indices,idx_to_char,char_to_idx,is_random_iter,
                        num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,
                        pred_len,prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:    #相邻采样，epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size,num_hiddens,ctx)
        l_sum,n,start = 0.0,0,time.time()
        data_iter = data_iter_fn(corpus_indices,batch_size,num_steps,ctx)
        for X,Y in data_iter:     #随机采样，在每个小批量更新前初始化隐藏状态
            if is_random_iter:
                state = init_rnn_state(batch_size,num_hiddens,ctx)
            else:    # 否则需要使⽤detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X,vocab_size)
                # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
                (outputs,state) = rnn(inputs,state,params)
                # 拼接之后形状为(num_steps * batch_size, vocab_size)
                outputs = nd.concat(*outputs,dim = 0)
                # Y的形状是(batch_size, num_steps)，转置后再变成⻓度为
                # batch * num_steps 的向量，这样跟输出的⾏⼀⼀对应
                y = Y.T.reshape((-1,))
                # 使⽤交叉熵损失计算平均分类误差
                l = loss(outputs,y).mean()
            l.backward()
            grad_clipping(params,clipping_theta,ctx)
            d2l.sgd(params,lr,1)
            l_sum+=l.asscalar() * y.size
            n+=y.size
        if (epoch+1) % pred_period ==0 :
            print('epoch %d, perplexity %f, time %.2f sec' % (
                    epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                        prefix, pred_len, rnn, params, init_rnn_state,
                        num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [23]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '笑了']

In [24]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                        vocab_size, ctx, corpus_indices, idx_to_char,
                        char_to_idx, True, num_epochs, num_steps, lr,
                        clipping_theta, batch_size, pred_period, pred_len,
                        prefixes)

epoch 50, perplexity 66.469914, time 0.27 sec
 - 分开 我想要这 你有我 别子我  你想我 别子我  你你 我想要再不 我不要再想 我不 我不 我不 我不
 - 笑了 我想要再 你有我 别子我  你想我 别子我  你你 我想要再不 我不要再想 我不 我不 我不 我不
epoch 100, perplexity 10.270906, time 0.27 sec
 - 分开 不颗我抬起头 有话去对医药 说你都很有  后悔你的我有多 就多 是 再血我遇多你 说知去的太是 像
 - 笑了 我不要再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承暴我已无处可躲 我不要再想 我不 我
epoch 150, perplexity 2.883388, time 0.25 sec
 - 分开 爱子在人留 不场掌斗 全家怕日出 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋
 - 笑了一碗悲粥 配上几斤的牛 它在安老斑鸠 平常话不多 除非都乌的喝了 我说店小二坦三著 别想我有别得到 
epoch 200, perplexity 1.580783, time 0.25 sec
 - 分开 一直在步三在的只斑鸠 印地安老斑鸠 腿短毛不多 几天都没有喝水也能活 脑袋瓜有一点秀逗 猎物死了它
 - 笑了一碗都粥 配上几斤 是谁在练太极 风生水起 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 习武我有
epoch 250, perplexity 1.305877, time 0.25 sec
 - 分开 那子她娘三仪的母斑鸠 印仔红蕃 在小镇 背对背决斗 一只灰狼 问候完空  偷有梦夫  一了痛夫落 
 - 笑了一碗热粥 配上几斤的玄 印地安老斑鸠 平常话不多 除非是乌鸦抢了它的窝 它在灌木丛旁邂逅 一只令它心


In [25]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                        vocab_size, ctx, corpus_indices, idx_to_char,
                        char_to_idx, False, num_epochs, num_steps, lr,
                        clipping_theta, batch_size, pred_period, pred_len,
                        prefixes)


epoch 50, perplexity 62.487440, time 0.25 sec
 - 分开 我想要再想 我不要再想 我不能这想 我不要再想 我不能这想 我不能这想 我不能这想 我不能这想 我
 - 笑了 我想要再想 我不要再想 我不能这想 我不要再想 我不能这想 我不能这想 我不能这想 我不能这想 我
epoch 100, perplexity 7.344889, time 0.25 sec
 - 分开 我想要再想你 你你想这 你天了沙截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快
 - 笑了 我说要这样 我不要再想 我不能再想 我不 我不 我不要 爱情走的太快就像龙卷 我不能再想 我不要再
epoch 150, perplexity 2.115241, time 0.25 sec
 - 分开 我给想再样你 你静那 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的
 - 笑了你都错 别想怎么 你来得难的溪边 默只就待远 说你着始些 我不能再想 我不能再想 我不 我不 我不能
epoch 200, perplexity 1.286326, time 0.26 sec
 - 分开 我说想这样你 爱静不是不人 漂分都靠板球 得分都靠多 除非过乌鸦抢了它的窝 它在灌木丛旁邂逅 一只
 - 笑了 我说儿 爱情走著看着就 说里手著我后棒 去教就跟已经怎么我 上海一九四三 泛黄的春联还残宙物了有弥
epoch 250, perplexity 1.203464, time 0.27 sec
 - 分开 我说想这样布这样的心不  为了我我想爸 那么凶 如果真 我属于这样牵 你的人美主 你家的美的字了 
 - 笑了 我说到 如情走著我想要我 我和不口我 你都寄红 不是完美 偷偷出容 没一秒钟 不敢不同 你不懂 连
